In [ ]:
from env import generate_url

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder 

In [ ]:
url = generate_url()

In [ ]:
list(pd.read_sql('SHOW DATABASES;',url)['Database'])

In [ ]:
dburl = generate_url('mall_customers')

In [ ]:
pd.read_sql("SHOW TABLES", dburl)

## Grabs DF

In [ ]:
df = pd.read_sql("SELECT * FROM customers", dburl)

In [ ]:
df.describe()

In [ ]:
sns.distplot(df.annual_income)

In [ ]:
q3 = df.annual_income.quantile(q=.75)

In [ ]:
q1 = df.annual_income.quantile(q=.25)

In [ ]:
iqr = q3 - q1 

In [ ]:
iqr_scaled = iqr * 1.5

In [ ]:
iqr_scaled - q1

In [ ]:
outlier_floor = iqr_scaled - q1

In [ ]:
outlier_ciel = iqr_scaled + q3

In [ ]:
def detect_outliers(df, col = 'annual_income'):
    q3 = df[col].quantile(q=.75)
    q1 = df[col].quantile(q=.25)
    iqr = q3 - q1
    iqr_scaled = iqr*1.5
    outlier_floor = iqr_scaled - q1
    outlier_ciel = iqr_scaled + q3
    outlier_df = pd.concat([df[df[col] > outlier_ciel],df[df[col] < outlier_floor]])
    return outlier_df

In [ ]:
df = pd.concat([df[df['annual_income'] > outlier_ciel], df[df['annual_income'] < outlier_floor]])

In [ ]:
detect_outliers(df)

In [ ]:
df.info()

In [ ]:
def encode_gender(df):
    encoder = LabelEncoder()
    encoder.fit(df.gender)
    return df.assign(gender = encoder.transform(df.gender))

In [ ]:
encode_gender(df).info()

In [ ]:
oldf = detect_outliers(df)
oldf

In [ ]:
for i in oldf.index:
    df.drop(i)

In [ ]:
def drop_outliers(df):
    outliers = detect_outliers(df)
    dfc = df.copy()
    for i in outliers.index:
        dfc.drop(i,inplace=True)
    return dfc 

In [ ]:
drop_outliers(df)

In [ ]:
df

In [ ]:
import prepare_mall

In [ ]:
df.info()

In [ ]:
prepare_mall.drop_outliers(df)

In [ ]:
def drop_outliers(df):
    outliers = detect_outliers(df)
    dfc = df.copy()
    for i in outliers.index:
        dfc.drop(i,inplace=True)
    return dfc

In [ ]:
drop_outliers(df)

In [ ]:
df

##  ZILLOW DATA ENGINEERING

In [ ]:
dburl = generate_url('zillow')
dfz = pd.read_sql('SELECT * FROM properties_2016 JOIN predictions_2016 USING (parcelid)',dburl)
tables = pd.read_sql('SHOW TABLES',dburl)
tables.drop(4,inplace=True)
tables.drop(5,inplace=True)
tables.drop(6,inplace=True)
tables.drop(7,inplace=True)
tables.drop(11,inplace=True)

type_dfs = []
for i in list(tables.Tables_in_zillow):
    type_dfs.append(pd.read_sql('SELECT * FROM '+i,dburl))
    
dfz2 = dfz
for i in type_dfs:
    dfz2 = pd.merge(dfz2,i,how="left")
    


In [ ]:
dfz.shape

In [ ]:
tables = pd.read_sql('SHOW TABLES',dburl)

In [ ]:

tables

In [ ]:
tables.drop(4,inplace=True)
tables.drop(5,inplace=True)
tables.drop(6,inplace=True)
tables.drop(7,inplace=True)
tables.drop(11,inplace=True)


In [ ]:
list(tables.Tables_in_zillow)

In [ ]:
type_dfs = []
for i in list(tables.Tables_in_zillow):
    type_dfs.append(pd.read_sql('SELECT * FROM '+i,dburl))

In [ ]:
list(type_dfs[0])[0]

In [ ]:
dfz2 = dfz
for i in type_dfs:
    dfz2 = pd.merge(dfz2,i,how="left")

In [ ]:
dfz2

In [2]:
import acquire_zillow

In [3]:
dfz = acquire_zillow.get_2016_properties()

In [4]:
import pandas as pd 
dfz.shape

(90275, 69)

In [5]:
dfz2 = acquire_zillow.get_2017_properties()

In [6]:
dfz2.shape

(77613, 69)

In [9]:
def all_zillow():
    
    return pd.concat([dfz,dfz2])
    

In [16]:
dfz3 = all_zillow()

In [14]:
dfz.head()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,11016594,2931667,1.0,NaN,NaN,2.0,3.0,NaN,4.0,2.0,...,0,0.0276,2016-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
1,14366692,475764,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,1,-0.1684,2016-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,12098116,2444575,1.0,NaN,NaN,3.0,2.0,NaN,4.0,3.0,...,2,-0.0040,2016-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
3,12643413,1584730,1.0,NaN,NaN,2.0,2.0,NaN,4.0,2.0,...,3,0.0218,2016-01-02,Central,NaN,NaN,Central,Condominium,NaN,NaN
4,14432541,2926916,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,...,4,-0.0050,2016-01-02,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN


In [15]:
dfz2.head()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,1727539,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,17052889,1387261,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,14186244,11677,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,12177905,2288172,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,10887214,1970746,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,4,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [17]:
dfz3.head()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,11016594,2931667,1.0,NaN,NaN,2.0,3.0,NaN,4.0,2.0,...,0,0.0276,2016-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
1,14366692,475764,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,1,-0.1684,2016-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,12098116,2444575,1.0,NaN,NaN,3.0,2.0,NaN,4.0,3.0,...,2,-0.0040,2016-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
3,12643413,1584730,1.0,NaN,NaN,2.0,2.0,NaN,4.0,2.0,...,3,0.0218,2016-01-02,Central,NaN,NaN,Central,Condominium,NaN,NaN
4,14432541,2926916,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,...,4,-0.0050,2016-01-02,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN


In [20]:
dfz3.drop('id',axis=1).shape

(167888, 67)